下载需要的包

pip install tensorflow

需要修改的部分为 读取数据的文件、根据原始数据标准化的公式【需要自己把握】、y预测部分的文件地址为训练参数所在的文件夹

In [ ]:
import pandas as pd
import numpy as np
import math
import os
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()   

In [ ]:
# 读取数据
df = pd.read_excel("C:/Users/133/Desktop/pop_data.xlsx")
# pd.read_csv("")
df.head()
data = df["UK"][0:59]

In [ ]:
# 绘制人口变化趋势图
plt.figure(figsize=(16,8))
plt.plot(data)
plt.show()

In [ ]:
#normalize_data=(data-np.mean(data))/np.std(data)  #标准化
#normalize_data=normalize_data[:,np.newaxis]       #增加维度
#需要将数据的尺度变小，由于数据是一直递增的，做非线性化
#神经网络需要数据变化幅度在一定小的范围内(大致在0-1)，数据小的变化可能影响网络
normalize_data=((np.log(data)/np.log(10)-6.584)/5)*100   
normalize_data=normalize_data[:,np.newaxis] 

In [ ]:
# 生成训练集
# 设置常量
time_step=20      #时间步
rnn_unit=10       #hidden layer units
batch_size=30     #每一批次训练多少个样例
input_size=1      #输入层维度
output_size=1     #输出层维度
lr=0.0006         #学习率

In [ ]:
train_x,train_y=[],[]   #训练集
for i in range(len(normalize_data)-time_step-1):
    x=normalize_data[i:i+time_step]
    y=normalize_data[i+1:i+time_step+1]
    train_x.append(x.tolist())  #将数组转化成列表
    train_y.append(y.tolist())

In [ ]:
#定义神经网络变量
X=tf.placeholder(tf.float32, [None,time_step,input_size])    #每批次输入网络的tensor/定义placeholder
Y=tf.placeholder(tf.float32, [None,time_step,output_size])   #每批次tensor对应的标签
#输入层、输出层权重、偏置
weights={
         'in':tf.Variable(tf.random_normal([input_size,rnn_unit])),
         'out':tf.Variable(tf.random_normal([rnn_unit,1]))
         }
biases={
        'in':tf.Variable(tf.constant(0.1,shape=[rnn_unit,])),
        'out':tf.Variable(tf.constant(0.1,shape=[1,]))
        }

In [ ]:
#定义LSTM神经网络变量
def lstm(batch):      #参数：输入网络批次数目
    w_in=weights['in']
    b_in=biases['in']
    input=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in #表示矩阵乘法
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)  #定义单个基本的LSTM单元
    init_state=cell.zero_state(batch,dtype=tf.float32)  #这个函数用于返回全0的state tensor
    #dynamic_rnn 用于创建由RNNCell细胞指定的循环神经网络，对inputs进行动态展开
    #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)
    #函数的作用是将tensor变换为参数shape的形式。 	
    output=tf.reshape(output_rnn,[-1,rnn_unit]) 
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out #表示矩阵乘法
    return pred,final_states


In [ ]:
#训练模型
def train_lstm():
    global batch_size
    pred,_=lstm(batch_size) #调用的构建的lstm变量
    #损失函数 平均平方误差(MSE)
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    #实现梯度下降算法的优化器，优化损失函数
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
    #保存和恢复模型的方法；方法返回checkpoint文件的路径。可以直接传给restore() 进行调用
    saver=tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #重复训练1000次
        step=0
        for i in range(2000):
            start=0
            end=start+batch_size
            while(end<len(train_x)):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[start:end],Y:train_y[start:end]})
                start+=batch_size
                end=start+batch_size
                #每10步保存一次参数
                if step%100==0:
                    print(i,step,loss_)
                    print("保存模型：",saver.save(sess,'pop.model'))
                step+=1

In [ ]:
with tf.variable_scope('train'):
    train_lstm()

In [ ]:
# 预测prediction()
with tf.variable_scope('train',reuse=True):
    pred,_=lstm(1)      #预测时只输入[1,time_step,input_size]的测试数据
    saver=tf.train.Saver(tf.global_variables())
    with tf.Session() as sess:
        #模型参数恢复
        #module_file = "pop.model"
        #module_file = tf.train.latest_checkpoint(base_path+'module2/')
        #saver.restore(sess, module_file) 
        saver.restore(sess,"C:/Users/133/Documents/Python Scripts/Untitled Folder/"+"pop.model")
 
        #取训练集最后一行为测试样本。shape=[1,time_step,input_size]
        prev_seq=train_x[-1]
        predict=[]
        #得到之后100个预测结果
        for i in range(26):
            next_seq=sess.run(pred,feed_dict={X:[prev_seq]})
            predict.append(next_seq[-1])
            #每次得到最后一个时间步的预测结果，与之前的数据加在一起，形成新的测试样本
            prev_seq=np.vstack((prev_seq[1:],next_seq[-1]))  
            #以折线图表示结果  

In [ ]:
#以折线图表示标准化结果  
plt.figure()
plt.plot(list(range(len(normalize_data))), normalize_data, color='b')
plt.plot(list(range(len(normalize_data), len(normalize_data) + len(predict))), predict, color='r')
plt.show()

In [ ]:
#返回数据计算
data_normalize = np.array(predict).squeeze()
data_pred = np.power(10,data_normalize/20+6.584)

#以折线图表示实际结果  
plt.figure()
plt.plot(range(len(data)), data, color='b')
plt.plot(range(len(data), len(data) + len(data_pred)), data_pred, color='r')
plt.show()

In [ ]:
# 结果导出
writer = pd.ExcelWriter("Y_结果导出.xlsx")
data_pred = pd.DataFrame(data_pred)
data_pred.to_excel(writer,"Yhat")
writer.save()